In [1]:
# Pandas dibutuhkan untuk manipulasi dan transformasi data
import pandas as pd

import numpy as np
# Sklearn digunakan untuk segala hal yang berhubungan dengan machine learning modeling
# train_test_split : untuk membagi data train menjadi data untuk training dan validasi
from sklearn.model_selection import train_test_split

# DecisionTreeClassifier : contoh model yang digunakan disini yaitu Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder

In [2]:
# Data Preparation
df = pd.read_csv('sampling_healtkathon2022/sampling_healtkathon2022.csv')

In [20]:
df.head

<bound method NDFrame.head of                 id  id_peserta  dati2 typefaskes  usia jenkel  pisat  \
0           165666         486     17         KL    48      P    1.0   
1          1010828         520     17          A    63      L    1.0   
2           166042         523     17         KL    53      P    1.0   
3           168937         549     17         KL    54      P    1.0   
4          1005899         549     17          A    53      P    1.0   
...            ...         ...    ...        ...   ...    ...    ...   
11401877   9983563     8527704    301          B     0      P    4.0   
11401878  11053870     8527709    233          A     0      P    4.0   
11401879   7461049     8527751    118          B     0      L    4.0   
11401880   1075162     8527853    101         SC     0      L    5.0   
11401881    102794     8527876     12         SD     0      P    4.0   

                         tgldatang                 tglpulang  jenispel  ...  \
0         2018-07-25T17:00

In [82]:
rand_processed = df

In [83]:
# Bikin labelencoder
labelencoder = LabelEncoder()

# Ngelabel encoding
rand_processed['jenkel_cat'] = labelencoder.fit_transform(rand_processed['jenkel'])

rand_processed['politujuan_cat'] = labelencoder.fit_transform(rand_processed['politujuan'])
rand_processed['typefaskes_cat'] = labelencoder.fit_transform(rand_processed['typefaskes'])
rand_processed['cbg_cat'] = labelencoder.fit_transform(rand_processed['cbg'])
rand_processed['diagfktp_cat'] = labelencoder.fit_transform(rand_processed['diagfktp'])

rand_processed['kdsa_cat'] = labelencoder.fit_transform(rand_processed['kdsa'])
rand_processed['kdsp_cat'] = labelencoder.fit_transform(rand_processed['kdsp'])
rand_processed['kdsr_cat'] = labelencoder.fit_transform(rand_processed['kdsr'])
rand_processed['kdsi_cat'] = labelencoder.fit_transform(rand_processed['kdsi'])
rand_processed['kdsd_cat'] = labelencoder.fit_transform(rand_processed['kdsd'])

# Drop semua yang gak dibutuhin
rand_processed = rand_processed.drop(columns=['id'])
rand_processed = rand_processed.drop(columns=['id_peserta'])

rand_processed = rand_processed.drop(columns=['jenkel'])

rand_processed = rand_processed.drop(columns=['politujuan'])
rand_processed = rand_processed.drop(columns=['typefaskes'])
rand_processed = rand_processed.drop(columns=['cbg'])
rand_processed = rand_processed.drop(columns=['diagfktp'])

rand_processed = rand_processed.drop(columns=['kdsa'])
rand_processed = rand_processed.drop(columns=['kdsp'])
rand_processed = rand_processed.drop(columns=['kdsr'])
rand_processed = rand_processed.drop(columns=['kdsi'])
rand_processed = rand_processed.drop(columns=['kdsd'])

# Drop semua yang gak dibutuhin
rand_processed['biaya_bagi100'] = rand_processed['biaya']/100
rand_processed = rand_processed.drop(columns=['biaya'])

rand_processed

,dati2,usia,pisat,tgldatang,tglpulang,jenispel,jenispulang,kelasrawat,label,jenkel_cat,politujuan_cat,typefaskes_cat,cbg_cat,diagfktp_cat,kdsa_cat,kdsp_cat,kdsr_cat,kdsi_cat,kdsd_cat,biaya_bagi100
0,17,48,1.0,2018-07-25T17:00:00.000Z,2018-07-25T17:00:00.000Z,2,1.0,3,0,1,224,16,825,5472,0,0,0,3,11,1843.0
1,17,63,1.0,2019-05-27T17:00:00.000Z,2019-05-30T17:00:00.000Z,1,1.0,1,0,0,261,0,106,11656,0,0,0,3,11,106284.0
2,17,53,1.0,2019-07-16T17:00:00.000Z,2019-07-16T17:00:00.000Z,2,1.0,3,0,1,161,16,827,2210,0,0,0,3,11,1873.0
3,17,54,1.0,2019-10-17T17:00:00.000Z,2019-10-17T17:00:00.000Z,2,1.0,3,0,1,178,16,827,3877,0,0,0,3,11,1873.0
4,17,53,1.0,2018-04-18T17:00:00.000Z,2018-04-18T17:00:00.000Z,2,1.0,3,0,1,218,0,827,7637,0,0,0,3,11,3816.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11401877,301,0,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,1.0,3,0,1,261,1,783,15143,0,0,0,3,11,36451.0
11401878,233,0,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,1.0,2,0,1,261,0,772,10671,0,0,0,3,11,122732.0
11401879,118,0,4.0,2020-12-30T17:00:00.000Z,2021-01-06T17:00:00.000Z,1,1.0,3,0,0,261,1,762,10540,0,0,0,3,11,40990.0
11401880,101,0,5.0,2020-12-29T17:00:00.000Z,2020-12-31T17:00:00.000Z,1,1.0,3,0,0,261,24,762,10421,0,0,0,3,11,39924.0


In [84]:
# Isi semua na
no_na = rand_processed
no_na['biaya_bagi100'] = no_na['biaya_bagi100'].fillna(no_na['biaya_bagi100'].mean())
no_na = no_na.fillna(0)

no_na

,dati2,usia,pisat,tgldatang,tglpulang,jenispel,jenispulang,kelasrawat,label,jenkel_cat,politujuan_cat,typefaskes_cat,cbg_cat,diagfktp_cat,kdsa_cat,kdsp_cat,kdsr_cat,kdsi_cat,kdsd_cat,biaya_bagi100
0,17,48,1.0,2018-07-25T17:00:00.000Z,2018-07-25T17:00:00.000Z,2,1.0,3,0,1,224,16,825,5472,0,0,0,3,11,1843.0
1,17,63,1.0,2019-05-27T17:00:00.000Z,2019-05-30T17:00:00.000Z,1,1.0,1,0,0,261,0,106,11656,0,0,0,3,11,106284.0
2,17,53,1.0,2019-07-16T17:00:00.000Z,2019-07-16T17:00:00.000Z,2,1.0,3,0,1,161,16,827,2210,0,0,0,3,11,1873.0
3,17,54,1.0,2019-10-17T17:00:00.000Z,2019-10-17T17:00:00.000Z,2,1.0,3,0,1,178,16,827,3877,0,0,0,3,11,1873.0
4,17,53,1.0,2018-04-18T17:00:00.000Z,2018-04-18T17:00:00.000Z,2,1.0,3,0,1,218,0,827,7637,0,0,0,3,11,3816.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11401877,301,0,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,1.0,3,0,1,261,1,783,15143,0,0,0,3,11,36451.0
11401878,233,0,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,1.0,2,0,1,261,0,772,10671,0,0,0,3,11,122732.0
11401879,118,0,4.0,2020-12-30T17:00:00.000Z,2021-01-06T17:00:00.000Z,1,1.0,3,0,0,261,1,762,10540,0,0,0,3,11,40990.0
11401880,101,0,5.0,2020-12-29T17:00:00.000Z,2020-12-31T17:00:00.000Z,1,1.0,3,0,0,261,24,762,10421,0,0,0,3,11,39924.0


In [85]:
no_na.describe()
#rand_processed.isna().sum()

,dati2,usia,pisat,jenispel,jenispulang,kelasrawat,label,jenkel_cat,politujuan_cat,typefaskes_cat,cbg_cat,diagfktp_cat,kdsa_cat,kdsp_cat,kdsr_cat,kdsi_cat,kdsd_cat,biaya_bagi100
count,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07,1.140188e+07
mean,1.825299e+02,3.705690e+01,2.498162e+00,1.645544e+00,1.094593e+00,2.784695e+00,1.375992e-02,5.404174e-01,1.986724e+02,1.228583e+01,6.482845e+02,7.149877e+03,6.350909e-01,2.766383e-01,1.066567e-01,3.015741e+00,1.096250e+01,2.146849e+04
std,1.061352e+02,2.292188e+01,1.390889e+00,4.783482e-01,5.131585e-01,5.511473e-01,1.164929e-01,4.983724e-01,5.577047e+01,1.061318e+01,2.797094e+02,4.550560e+03,4.717502e+00,1.827107e+00,7.898754e-01,1.431721e-01,5.606337e-01,5.194357e+04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.140000e+02,1.800000e+01,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,1.580000e+02,2.000000e+00,4.540000e+02,3.447000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.100000e+01,1.904000e+03
50%,1.590000e+02,3.900000e+01,3.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.820000e+02,9.000000e+00,8.080000e+02,5.446000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.100000e+01,2.906000e+03
75%,2.280000e+02,5.600000e+01,4.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,2.610000e+02,2.400000e+01,8.270000e+02,1.157700e+04,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.100000e+01,2.982100e+04
max,5.280000e+02,1.100000e+02,5.000000e+00,2.000000e+00,5.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,2.610000e+02,2.500000e+01,1.033000e+03,1.701200e+04,3.600000e+01,1.300000e+01,6.000000e+00,4.000000e+00,1.200000e+01,4.016966e+06


In [5]:
Pred = no_na.drop(columns=['label'])


In [7]:
Pred = Pred.select_dtypes(exclude=['object'])

In [9]:
# Pembuatan data training

satu = no_na.loc[no_na['label'] == 1] # Pengambilan yang labelnya 1
nolfull = no_na.loc[no_na['label'] == 0] # Pengambilan yang label 0

nol = nolfull.sample(n=len(satu)) # Pengambilan label 0 sejumlah banyaknya label 1



In [10]:
Train = nol.append(satu)
Train = Train.sample(frac = 1)
Train = Train.select_dtypes(exclude=['object'])

C:\Users\faris\AppData\Local\Temp\ipykernel_21016\4069620333.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Train = nol.append(satu)


In [14]:
Train.info()
Pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123692 entries, 7387923 to 4712934
Data columns (total 36 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id_peserta   123692 non-null  int64  
 1   dati2        123692 non-null  int64  
 2   usia         123692 non-null  int64  
 3   jenkel       123692 non-null  float64
 4   pisat        123692 non-null  float64
 5   jenispel     123692 non-null  int64  
 6   biaya        123692 non-null  float64
 7   jenispulang  123692 non-null  float64
 8   kelasrawat   123692 non-null  int64  
 9   label        123692 non-null  int64  
 10  A            123692 non-null  float64
 11  B            123692 non-null  float64
 12  C            123692 non-null  float64
 13  D            123692 non-null  float64
 14  GD           123692 non-null  float64
 15  HD           123692 non-null  float64
 16  I1           123692 non-null  float64
 17  I2           123692 non-null  float64
 18  I3           1236

In [12]:
print(len(nol))
print(len(satu))
print(len(Train))

61846
61846
123692


In [13]:
X = Train.drop(columns=['label'])
y = Train.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# svc = SVC(probability = True)
# svc = svc.fit(X_train, y_train)

In [ ]:
# Hasilpred = svc.predict(Pred)

In [ ]:
# rf = RandomForestClassifier(random_state = 1)
# rf = rf.fit(X_train, y_train)

In [ ]:
# Hasilpred = rf.predict(Pred)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
Hasilpred = clf.predict(Pred)

In [ ]:
# Evaluation

y_true = rand_processed.label
y_pred = Hasilpred


def check(true,pred)  :
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    Accuracy = (tn+tp) / (tn+fp+tp+fn)
    Precision = tp/(tp+fp)
    Recall = tp/(tp+fn)
    Specifity = tn/(tn+fp)  

    print("Accuracy    :", Accuracy, "\nPrecision   :", Precision, "\nRecall      :", Recall, "\nSpecifity   :", Specifity)


check(y_true,y_pred)

